In [1]:
%who

Interactive namespace is empty.


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import warnings
warnings.filterwarnings("ignore")

/Users/hanlinzhuang/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
data = pd.read_csv('../../raw data/new/btc_hourly.csv')

In [4]:
data['vol_per_count'] = data['volume_traded']/data['trades_count']
data['roi'] = np.log(data['price_close']/data['price_close'].shift())*100
data['price_close_lag1'] = data['price_close'].shift(1)
data['price_close_lag2'] = data['price_close'].shift(2)
data['price_close_lag3'] = data['price_close'].shift(3)
data['volume_traded_lag1'] = data['volume_traded'].shift(1)
data['trades_count_lag1'] = data['trades_count'].shift(1)
data['vol_per_count_lag1'] = data['vol_per_count'].shift(1)
data['roi_lag1'] = data['roi'].shift(1)
data['roi_lag2'] = data['roi'].shift(2)
data['roi_lag3'] = data['roi'].shift(3)

In [5]:
data['roi'].describe()

count    2135.000000
mean        0.020387
std         3.020823
min       -17.963276
25%        -0.292303
50%         0.022231
75%         0.346353
max        17.713383
Name: roi, dtype: float64

In [6]:
std = data['roi'].std()
data['cut'] = pd.cut(data['roi'], 
                     bins=[-100, -3*std, -2*std, -std, 0, std, 2*std, 3*std, 100], 
                    labels=['-4', '-3', '-2', '-1', '+1', '+2', '+3', '+4'])
data['cut_lag1'] = data['cut'].shift(1)
data['cut_lag2'] = data['cut'].shift(2)
data['cut_lag3'] = data['cut'].shift(3)

In [7]:
data.head()

,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count,...,volume_traded_lag1,trades_count_lag1,vol_per_count_lag1,roi_lag1,roi_lag2,roi_lag3,cut,cut_lag1,cut_lag2,cut_lag3
0,2019-02-01T00:00:00.0000000Z,2019-02-01T01:00:00.0000000Z,2019-02-01T00:00:00.0000000Z,2019-02-01T00:59:58.6240000Z,3413.69,4801.89,3302.01111,3410.62,16176.11220,5239,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-02-01T01:00:00.0000000Z,2019-02-01T02:00:00.0000000Z,2019-02-01T01:00:00.0000000Z,2019-02-01T01:59:58.0000000Z,3496.00,5575.47,2197.12000,3387.20,17787.21738,8078,...,16176.11220,5239.0,3.087634,NaN,NaN,NaN,-1,NaN,NaN,NaN
2,2019-02-01T02:00:00.0000000Z,2019-02-01T03:00:00.0000000Z,2019-02-01T02:00:00.0000000Z,2019-02-01T02:59:59.1200000Z,3476.00,4055.97,2191.11000,3388.21,14935.60829,4863,...,17787.21738,8078.0,2.201933,-0.689047,NaN,NaN,+1,-1,NaN,NaN
3,2019-02-01T03:00:00.0000000Z,2019-02-01T04:00:00.0000000Z,2019-02-01T03:00:00.0000000Z,2019-02-01T03:59:59.4890000Z,3480.70,5519.47,2175.06000,3395.34,15145.67548,4217,...,14935.60829,4863.0,3.071275,0.029814,-0.689047,NaN,+1,+1,-1,NaN
4,2019-02-01T04:00:00.0000000Z,2019-02-01T05:00:00.0000000Z,2019-02-01T04:00:00.0000000Z,2019-02-01T04:59:57.0000000Z,3394.48,4055.93,3302.00001,3388.19,14572.54123,3339,...,15145.67548,4217.0,3.591576,0.210214,0.029814,-0.689047,-1,+1,+1,-1


In [8]:
data.columns

Index(['time_period_start', 'time_period_end', 'time_open', 'time_close',
       'price_open', 'price_high', 'price_low', 'price_close', 'volume_traded',
       'trades_count', 'vol_per_count', 'roi', 'price_close_lag1',
       'price_close_lag2', 'price_close_lag3', 'volume_traded_lag1',
       'trades_count_lag1', 'vol_per_count_lag1', 'roi_lag1', 'roi_lag2',
       'roi_lag3', 'cut', 'cut_lag1', 'cut_lag2', 'cut_lag3'],
      dtype='object')

In [9]:
data_ = data.dropna()
data_.head()

,time_period_start,time_period_end,time_open,time_close,price_open,price_high,price_low,price_close,volume_traded,trades_count,...,volume_traded_lag1,trades_count_lag1,vol_per_count_lag1,roi_lag1,roi_lag2,roi_lag3,cut,cut_lag1,cut_lag2,cut_lag3
4,2019-02-01T04:00:00.0000000Z,2019-02-01T05:00:00.0000000Z,2019-02-01T04:00:00.0000000Z,2019-02-01T04:59:57.0000000Z,3394.48,4055.93,3302.00001,3388.19,14572.54123,3339,...,15145.67548,4217.0,3.591576,0.210214,0.029814,-0.689047,-1,+1,+1,-1
5,2019-02-01T05:00:00.0000000Z,2019-02-01T06:00:00.0000000Z,2019-02-01T05:00:02.0000000Z,2019-02-01T05:59:59.4910000Z,3478.40,4056.33,3322.40003,3394.02,12988.44231,3028,...,14572.54123,3339.0,4.364343,-0.210805,0.210214,0.029814,+1,-1,+1,+1
6,2019-02-01T06:00:00.0000000Z,2019-02-01T07:00:00.0000000Z,2019-02-01T06:00:00.3530000Z,2019-02-01T06:59:59.0000000Z,3394.02,4053.97,3302.56000,4051.75,11399.31467,3099,...,12988.44231,3028.0,4.289446,0.171920,-0.210805,0.210214,+4,+1,-1,+1
7,2019-02-01T07:00:00.0000000Z,2019-02-01T08:00:00.0000000Z,2019-02-01T07:00:00.0000000Z,2019-02-01T07:59:59.0000000Z,3698.00,4053.22,2952.63000,3712.00,17215.16712,6224,...,11399.31467,3099.0,3.678385,17.713383,0.171920,-0.210805,-3,+4,+1,-1
8,2019-02-01T08:00:00.0000000Z,2019-02-01T09:00:00.0000000Z,2019-02-01T08:00:02.0000000Z,2019-02-01T08:59:59.0000000Z,3470.80,4467.20,3210.80000,3354.49,14957.96840,4368,...,17215.16712,6224.0,2.765933,-8.757807,17.713383,0.171920,-4,-3,+4,+1


In [10]:
data_['cut'].groupby(data_['cut']).count()

cut
-4      35
-3      17
-2      45
-1     919
+1    1010
+2      60
+3       9
+4      37
Name: cut, dtype: int64

In [11]:
data_.shape

(2132, 25)

In [12]:
y = data_['price_close']
X = data_[['price_close_lag1', 'price_close_lag2', 'price_close_lag3', 
           'roi_lag1', 
           'volume_traded_lag1', 'trades_count_lag1', 'vol_per_count_lag1']]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [14]:
mod_linear = LinearRegression()
mod_linear.fit(X_train, y_train);

In [15]:
print('coefficients:')
for i in range(len(X.columns)):
    print('{}: {}'.format(X.columns[i], mod_linear.coef_[i]))

coefficients:
price_close_lag1: 0.35716442815247873
price_close_lag2: 0.3304178594183203
price_close_lag3: 0.29576350476879615
roi_lag1: 0.7774691989418128
volume_traded_lag1: -0.0005278698074646435
trades_count_lag1: 0.0010965415600147862
vol_per_count_lag1: -0.5201979355786112


In [16]:
y_predicted = mod_linear.predict(X_test)
print('mse:')
print(mean_squared_error(y_test, y_predicted))

mse:
17818.638218153206


In [17]:
print('accuracy:')
print(mod_linear.score(X_test, y_test))

accuracy:
0.40047864488691404


### clf

In [18]:
# y_clf = data_['cut']
# X_clf = data_[['roi_lag1', 'roi_lag2', 'roi_lag3', 
#                'price_close_lag1', 
#                'volume_traded_lag1', 'trades_count_lag1', 'vol_per_count_lag1']]

In [19]:
y_clf = data_['cut']
X_clf = data_[['roi_lag1',  
               'price_close_lag1', 
               'volume_traded_lag1', 'trades_count_lag1', 'vol_per_count_lag1']]

In [20]:
data_[['cut', 'roi_lag1',  
               'price_close_lag1', 
               'volume_traded_lag1', 'trades_count_lag1', 'vol_per_count_lag1']].head()

,cut,roi_lag1,price_close_lag1,volume_traded_lag1,trades_count_lag1,vol_per_count_lag1
4,-1,0.210214,3395.34,15145.67548,4217.0,3.591576
5,+1,-0.210805,3388.19,14572.54123,3339.0,4.364343
6,+4,0.171920,3394.02,12988.44231,3028.0,4.289446
7,-3,17.713383,4051.75,11399.31467,3099.0,3.678385
8,-4,-8.757807,3712.00,17215.16712,6224.0,2.765933


In [21]:
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf, y_clf, test_size=0.2, random_state=0)

In [22]:
y_clf_test.shape

(427,)

In [23]:
# random forest

In [24]:
clf_rf = RandomForestClassifier(n_estimators=1000, max_depth=5, bootstrap=True, random_state=0)
clf_rf.fit(X_clf_train, y_clf_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [25]:
print('accuracy:')
print(clf_rf.score(X_clf_test, y_clf_test))

accuracy:
0.6182669789227166


In [26]:
y_clf_predicted = clf_rf.predict(X_clf_test)
confusionmatrix = pd.DataFrame(confusion_matrix(y_clf_test, y_clf_predicted, labels=['-4', '-3', '-2', '-1', '+1', '+2', '+3', '+4']), columns
=['Pred -4', 'Pred -3', 'Pred -2', 'Pred -1', 'Pred +1', 'Pred +2', 'Pred +3', 'Pred +4'],
index=['Actual -4', 'Actual -3', 'Actual -2', 'Actual -1', 'Actual +1', 'Actual +2', 'Actual +3', 'Actual +4'])
confusionmatrix

,Pred -4,Pred -3,Pred -2,Pred -1,Pred +1,Pred +2,Pred +3,Pred +4
Actual -4,7,0,0,0,0,0,0,0
Actual -3,0,0,1,2,1,0,0,0
Actual -2,0,0,5,3,1,0,0,0
Actual -1,0,0,0,78,103,0,0,0
Actual +1,0,0,0,31,174,0,0,0
Actual +2,0,0,0,1,12,0,0,0
Actual +3,0,0,0,0,2,0,0,0
Actual +4,0,0,0,1,5,0,0,0


In [27]:
y_clf_test.shape

(427,)

In [28]:
print("Average precision for the 8 intervals is - ", precision_score(y_clf_test, y_clf_predicted, average = None))

Average precision for the 8 intervals is -  [0.58389262 0.         0.         0.         0.67241379 0.83333333
 0.         1.        ]


In [29]:
print("Average recall for the 8 intervals is - ", recall_score(y_clf_test, y_clf_predicted, average = None))

Average recall for the 8 intervals is -  [0.84878049 0.         0.         0.         0.43093923 0.55555556
 0.         1.        ]


In [30]:
# boosting

In [31]:
clf_gb = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01, 
                                            max_depth=1, random_state=0)
clf_gb.fit(X_clf_train, y_clf_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=1, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=0,
             subsample=1.0, verbose=0, warm_start=False)

In [32]:
print('accuracy:')
print(clf_gb.score(X_clf_test, y_clf_test))

accuracy:
0.2873626739943522


In [33]:
y_clf_predicted = clf_gb.predict(X_clf_test)
y_clf_predicted = pd.DataFrame(y_clf_predicted)
y_clf_predicted['cut'] = pd.cut(y_clf_predicted[0], 
                     bins=[-100, -3*std, -2*std, -std, 0, std, 2*std, 3*std, 100], 
                    labels=['-4', '-3', '-2', '-1', '+1', '+2', '+3', '+4'])

In [34]:
# ____________

In [35]:
confusionmatrix = pd.DataFrame(confusion_matrix(y_clf_test, y_clf_predicted['cut'], labels=['-4', '-3', '-2', '-1', '+1', '+2', '+3', '+4']), columns
=['Pred -4', 'Pred -3', 'Pred -2', 'Pred -1', 'Pred +1', 'Pred +2', 'Pred +3', 'Pred +4'],
index=['Actual -4', 'Actual -3', 'Actual -2', 'Actual -1', 'Actual +1', 'Actual +2', 'Actual +3', 'Actual +4'])
confusionmatrix

,Pred -4,Pred -3,Pred -2,Pred -1,Pred +1,Pred +2,Pred +3,Pred +4
Actual -4,0,0,7,0,0,0,0,0
Actual -3,0,0,0,3,1,0,0,0
Actual -2,0,0,0,8,1,0,0,0
Actual -1,0,0,0,67,114,0,0,0
Actual +1,0,0,0,23,182,0,0,0
Actual +2,0,0,0,1,12,0,0,0
Actual +3,0,0,0,0,2,0,0,0
Actual +4,0,0,0,1,5,0,0,0


In [36]:
print("Average precision for the 8 intervals is - ", precision_score(y_clf_test, y_clf_predicted['cut'], average = None))

Average precision for the 8 intervals is -  [0.57413249 0.         0.         0.         0.65048544 0.
 0.         0.        ]


In [37]:
print("Average recall for the 8 intervals is - ", recall_score(y_clf_test, y_clf_predicted['cut'], average = None))

Average recall for the 8 intervals is -  [0.88780488 0.         0.         0.         0.37016575 0.
 0.         0.        ]


In [38]:
# Adaboosting

In [39]:
clf_adab = AdaBoostClassifier(n_estimators=1000, learning_rate=1, algorithm='SAMME')
clf_adab.fit(X_clf_train,y_clf_train)

AdaBoostClassifier(algorithm='SAMME', base_estimator=None, learning_rate=1,
          n_estimators=1000, random_state=None)

In [40]:
print('accuracy:')
print(clf_adab.score(X_clf_test, y_clf_test))

accuracy:
0.5878220140515222


In [41]:
y_clf_predicted = clf_adab.predict(X_clf_test)
confusionmatrix = pd.DataFrame(confusion_matrix(y_clf_test, y_clf_predicted, labels=['-4', '-3', '-2', '-1', '+1', '+2', '+3', '+4']), columns
=['Pred -4', 'Pred -3', 'Pred -2', 'Pred -1', 'Pred +1', 'Pred +2', 'Pred +3', 'Pred +4'],
index=['Actual -4', 'Actual -3', 'Actual -2', 'Actual -1', 'Actual +1', 'Actual +2', 'Actual +3', 'Actual +4'])
confusionmatrix

,Pred -4,Pred -3,Pred -2,Pred -1,Pred +1,Pred +2,Pred +3,Pred +4
Actual -4,6,1,0,0,0,0,0,0
Actual -3,0,2,1,0,1,0,0,0
Actual -2,0,2,4,1,2,0,0,0
Actual -1,0,0,1,75,101,1,3,0
Actual +1,0,0,1,38,164,1,1,0
Actual +2,0,0,0,4,9,0,0,0
Actual +3,0,0,0,0,2,0,0,0
Actual +4,0,0,0,1,5,0,0,0


In [42]:
print("Average precision for the 8 intervals is - ", precision_score(y_clf_test, y_clf_predicted, average = None))

Average precision for the 8 intervals is -  [0.57746479 0.         0.         0.         0.6302521  0.57142857
 0.4        1.        ]


In [43]:
print("Average recall for the 8 intervals is - ", recall_score(y_clf_test, y_clf_predicted, average = None))

Average recall for the 8 intervals is -  [0.8        0.         0.         0.         0.41436464 0.44444444
 0.5        0.85714286]
